In [ ]:
from google.colab import drive
import datetime
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import datetime
ZED = []
def perc(y,x):
    z = ((y - x) / x) * 100
    return z

def percentage (value,percentvalue):
    z = value*((percentvalue/100))
    return z


def percentagetrade (value,percentvalue):
    z = value + value*((percentvalue/100))
    return z
#import matplotlib.pyplot as plt

import statistics

dfx = pd.DataFrame( columns = ["STOCK", "PERIOD","CANDLE",
                               "BUYTARGETPERCENT","SELLTARGETPERCENTT",
                               "BUYTARGETACHIEVED","SELLTARGETACHIEVED",
                               "PROFITABLETRADES","LOSSTRADES",
                               "TOTALTRADES","PNL","TIMEFRAME"])

STOCKS = ['NIFTY','BANKNIFTY','RELIANCE','HDFC','HDFCBANK','TATAMOTORS','TATASTEEL','ASHOKLEY','AXISBANK','AUROPHARMA','BHARATFORG','BPCL','BIOCON','BHARTIARTL','BANKBARODA','CIPLA','CADILAHC','DIVISLAB','DRREDDY','ESCORTS','GLENMARK','HINDALCO','HCLTECH','INFY','ICICIBANK','LUPIN','LT','MARUTI','SUNTV','SUNPHARMA','SBIN','BAJFINANCE','BATAINDIA','MGL','MINDTREE','JUBLFOOD','TVSMOTOR']
STOCKS = ['MOTHERSUMI','NATIONALUM','NMDC','NTPC','SAIL','SIEMENS','TATACHEM','TATAMOTORS','TATAPOWER','VEDL','ADANIENT','ADANIPORTS','BEL','BHARTIARTL','BHEL','BPCL','CUMMINSIND','GMRINFRA','GRASIM','L&TFH']

minutess = [5,10,15]
gaps = [30]
for gap in gaps:
  for instrument in STOCKS:
      for minutes in minutess:
          tradedbuy = 0
          tradedsell = 0
          targetbuy = 0
          targetsell = 0
          tradedsellz = 0
          tradedbuyz = 0

          stock = f"{instrument}-I.NFO"
          listt = []

          dz = "x"
          dfz = pd.DataFrame( columns = ["EXITPERCENTAGE", "POSITIONPRICE","EXITPRICE","PNL","POSITIONTIME","TRADE"])


          df2 = pd.read_csv(f"/content/drive/My Drive/reports/{gap}AAA{stock}FINAL{minutes}minexperimentDEC20195HL.csv")

          df2.dropna(subset = ["TRADE"], inplace=True)

          df2['TIME'] = df2['DATETIME']
          df2.loc[df2['TRADE'] == 'SELL', 'TRADEE'] = 'SELL'
          df2.loc[df2['TRADE'] == 'BUY', 'TRADEE'] = 'BUY'
          trade_signal = df2['TRADEE']
          ts_shifted = trade_signal.shift(1)
          df2['TRADEE'] = np.where(trade_signal != ts_shifted, trade_signal, 0)
          df2.loc[df2['TRADEE'] == 'SELL', 'TRADED'] = df2['Open']
          df2.loc[df2['TRADEE'] == 'BUY', 'TRADED'] = df2['Open']
          df2.loc[df2['TRADEE'] == 'SELL', 'TRADEDTIME'] = df2['DATETIME']
          df2.loc[df2['TRADEE'] == 'BUY', 'TRADEDTIME'] = df2['DATETIME']
          df2 = (df2.ffill())
          df2.loc[df2['TRADE'] == 'SELL', 'TARGET'] = perc(df2['Low'], df2['TRADED'])
          df2.loc[df2['TRADE'] == 'BUY', 'TARGET'] = perc(df2['High'], df2['TRADED'])


          df2['DATETIME'] = df2['DATETIME'].astype(str)

          #print(df2.to_csv("dataxlxxx.csv"))
          highes = []
          lows = []
          percenttargetlistbuysell = df2['TRADEDTIME'].to_list()
          percenttargetlistbuysell = list(dict.fromkeys(percenttargetlistbuysell))
          #percenttargetlistbuysell = list(set(percenttargetlistbuysell))


          for  targetbuy  in np.arange(1,10):
              #for  targetsell in np.arange(1,10):

                  targetsell  = -1*targetbuy
                  tradedbuy = 0
                  tradedsell = 0

                  for i in percenttargetlistbuysell:
                      df4 = df2.loc[(df2['TRADEDTIME'] == i)]
                      minimum = df4['TARGET'].min()
                      maximum  = df4['TARGET'].max()

                      traded = df4['TRADED'].iloc[0]
                      tradedtimme = df4['TRADEDTIME'].iloc[0]


                      if df4['TRADE'].iloc[0] == 'SELL' :
                          #print(tradedtimme, df4['TRADE'].iloc[0], minimum)
                          if targetsell < minimum:
                              tradedsell = traded

                          if tradedbuy!=0:
                              highes.append([perc(traded,tradedbuy),tradedbuy,traded,((percentage(tradedbuy,perc(traded,tradedbuy)))),tradedtimme,"BUYSELL"])
                              tradedsell = traded
                              #print(highes[-1])
                              to_append = highes[-1]
                              a_series = pd.Series(to_append, index=dfz.columns)
                              dfz = dfz.append(a_series, ignore_index=True)


                          if ((df4['TARGET']).min()) != 0 and  targetsell >= minimum :
                              lows.append([targetsell,traded,percentagetrade(traded,targetsell),-1*(percentage(traded,targetsell)),tradedtimme,"SELLTARGET"])
                              tradedbuy = 0
                              tradedsell = 0
                              #print(lows[-1])
                              to_append = lows[-1]
                              a_series = pd.Series(to_append, index=dfz.columns)
                              dfz = dfz.append(a_series, ignore_index=True)



                      if df4['TRADE'].iloc[0] == 'BUY':
                          #print(tradedtimme, df4['TRADE'].iloc[0], maximum)
                          if targetbuy > maximum:
                              tradedbuy = traded

                          if tradedsell!=0:
                              lows.append([perc(traded,tradedsell),tradedsell,traded,-1*(percentage(tradedsell,perc(traded,tradedsell))),tradedtimme,'SELLBUY'])
                              tradedbuy = traded
                              #print(lows[-1])
                              to_append = lows[-1]
                              a_series = pd.Series(to_append, index=dfz.columns)
                              dfz = dfz.append(a_series, ignore_index=True)



                          if ((df4['TARGET']).max()) != 0 and targetbuy <= maximum :
                              highes.append([targetbuy,traded,percentagetrade(traded,targetbuy),(percentage(traded,targetbuy)),tradedtimme,"BUYTARGET"])
                              tradedbuy = 0
                              tradedsell = 0


                              #print(highes[-1])
                              to_append = highes[-1]
                              a_series = pd.Series(to_append, index=dfz.columns)
                              dfz = dfz.append(a_series, ignore_index=True)


                  lows.sort()
                  highes.sort(reverse=True)
                  highadd = []
                  lowadd = []
                  XP = (dfz.loc[(dfz['TRADE'] == "BUYTARGET")])
                  buytargetachieved = XP.shape[0]
                  XP = (dfz.loc[(dfz['TRADE'] == "SELLTARGET")])
                  selltargetachieved = XP.shape[0]
                  XP = (dfz.loc[(dfz['PNL'] < 0)])
                  losstrades = XP.shape[0]
                  XP = (dfz.loc[(dfz['PNL'] > 0)])
                  profittrades = XP.shape[0]
                  XP = 0
                  totaltrades = dfz.shape[0]
                  
                  #experiment
                  total = dfz['PNL'].sum()
                  
                  #experiment
                  
                  

                  report = [stock, "fromDEC2019", minutes, targetbuy, targetsell, buytargetachieved, selltargetachieved,
                            profittrades, losstrades, totaltrades, total,gap]



                  PL = total
                  TOTAL = [PL,report]
                  
                  #print(dfz.to_csv(f"/content/drive/My Drive/reportss/NIFTYYY{stock}max{minutes}reportfrommayoptimizeparam.csv"))
                  dfz = pd.DataFrame( columns = ["EXITPERCENTAGE", "POSITIONPRICE","EXITPRICE","PNL","POSITIONTIME","TRADE"])

                  #print("SELLTARGET",count)


                  #print(TOTAL)
                  highes.clear()
                  lows.clear()
                  highadd.clear()
                  lowadd.clear()
                  listt.append(TOTAL)
                  #print(listt[-1],max(listt))



        #print('nextmin')
          print(max(listt),minutes)
          report = max(listt)[-1]
          to_append = report
          a_series = pd.Series(to_append, index=dfx.columns)
          dfx = dfx.append(a_series, ignore_index=True)
          #print(dfz)
          #print(dfz.to_csv(f"/content/drive/My Drive/reports/{stock}max{minutes}EXPERIMENTWITHOUTfromMAY.csv"))
          listt.clear()
          dfz = pd.DataFrame()
          #print(dfx)
#5minnifty
#10minnifty
#15minnifty
print(dfx)
df = dfx
idx = df.groupby(['STOCK'])['PNL'].transform(max) == df['PNL']
ZX = (df[idx])
print(ZX)
print(ZX.to_csv("/content/drive/My Drive/reportss/30MINUTEBREAK.csv"))



[2.011499999999941, ['MOTHERSUMI-I.NFO', 'fromDEC2019', 5, 3, -3, 20, 16, 44, 65, 109, 2.011499999999941, 30]] 5
[-7.239500000000017, ['MOTHERSUMI-I.NFO', 'fromDEC2019', 10, 3, -3, 20, 15, 40, 58, 99, -7.239500000000017, 30]] 10
[-29.027000000000015, ['MOTHERSUMI-I.NFO', 'fromDEC2019', 15, 2, -2, 21, 20, 44, 54, 98, -29.027000000000015, 30]] 15
[14.72700000000004, ['NATIONALUM-I.NFO', 'fromDEC2019', 5, 7, -7, 6, 4, 35, 48, 86, 14.72700000000004, 30]] 5
[14.941000000000022, ['NATIONALUM-I.NFO', 'fromDEC2019', 10, 7, -7, 6, 4, 35, 45, 80, 14.941000000000022, 30]] 10
[14.35250000000002, ['NATIONALUM-I.NFO', 'fromDEC2019', 15, 7, -7, 8, 3, 33, 42, 76, 14.35250000000002, 30]] 15
[-5.958000000000059, ['NMDC-I.NFO', 'fromDEC2019', 5, 4, -4, 14, 6, 39, 60, 100, -5.958000000000059, 30]] 5
[2.1199999999999406, ['NMDC-I.NFO', 'fromDEC2019', 10, 4, -4, 16, 6, 35, 56, 92, 2.1199999999999406, 30]] 10
[-12.145999999999999, ['NMDC-I.NFO', 'fromDEC2019', 15, 4, -4, 14, 6, 34, 51, 86, -12.14599999999999

In [ ]:
dfx

,STOCK,PERIOD,CANDLE,BUYTARGETPERCENT,SELLTARGETPERCENTT,BUYTARGETACHIEVED,SELLTARGETACHIEVED,PROFITABLETRADES,LOSSTRADES,TOTALTRADES,PNL,TIMEFRAME
0,MOTHERSUMI-I.NFO,fromDEC2019,5,3,-3,20,16,44,65,109,2.0115,30
1,MOTHERSUMI-I.NFO,fromDEC2019,10,3,-3,20,15,40,58,99,-7.2395,30
2,MOTHERSUMI-I.NFO,fromDEC2019,15,2,-2,21,20,44,54,98,-29.0270,30
3,NATIONALUM-I.NFO,fromDEC2019,5,7,-7,6,4,35,48,86,14.7270,30
4,NATIONALUM-I.NFO,fromDEC2019,10,7,-7,6,4,35,45,80,14.9410,30
5,NATIONALUM-I.NFO,fromDEC2019,15,7,-7,8,3,33,42,76,14.3525,30
6,NMDC-I.NFO,fromDEC2019,5,4,-4,14,6,39,60,100,-5.9580,30
7,NMDC-I.NFO,fromDEC2019,10,4,-4,16,6,35,56,92,2.1200,30
8,NMDC-I.NFO,fromDEC2019,15,4,-4,14,6,34,51,86,-12.1460,30
9,NTPC-I.NFO,fromDEC2019,5,9,-9,2,1,40,65,105,25.9210,30


In [ ]:
df = dfx
idx = df.groupby(['STOCK'])['PNL'].transform(max) == df['PNL']
ZX = (df[idx])
print(ZX)
print(ZX.to_csv("/content/drive/My Drive/reportss/30MINUTEBREAK.csv"))


               STOCK       PERIOD CANDLE  ... TOTALTRADES      PNL TIMEFRAME
0   MOTHERSUMI-I.NFO  fromDEC2019      5  ...         109   2.0115        30
4   NATIONALUM-I.NFO  fromDEC2019     10  ...          80  14.9410        30
7         NMDC-I.NFO  fromDEC2019     10  ...          92   2.1200        30
10        NTPC-I.NFO  fromDEC2019     10  ...         101  28.1200        30
13        SAIL-I.NFO  fromDEC2019     10  ...          94   7.5425        30
15     SIEMENS-I.NFO  fromDEC2019      5  ...          92  40.7730        30
20    TATACHEM-I.NFO  fromDEC2019     15  ...          93  35.2850        30
22  TATAMOTORS-I.NFO  fromDEC2019     10  ...          90  55.9000        30
26   TATAPOWER-I.NFO  fromDEC2019     15  ...          69  29.8735        30
28        VEDL-I.NFO  fromDEC2019     10  ...          96   8.8730        30

[10 rows x 12 columns]
None
